In [5]:
import re
from tqdm import tqdm
import numpy as np
import random
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
path_to_data = "/home/kirrog/projects/CheckerApp/dataset/lenta-ru-news.csv"

In [ ]:
def cut_text(text: str):
    d = ""
    try:
        d = " ".join(text[max(text.index(",\"") + 2, text.index("/,") + 2):text.index("\",")].split())
    except Exception as e:
        print(e)
    return d

In [ ]:
with open(path_to_data, "r") as f:
    data_l = f.read().split("https://")

In [ ]:
r = []
for l in data_l:
    d = cut_text(l)
    if d != "":
        r.append(d)

In [ ]:
len(data_l)

In [ ]:
data_l = None

In [ ]:
len(r)

In [ ]:
r[0]

In [ ]:
r = [re.sub('\n', f"", x) for x in r]

In [1]:
clean_text_path = "/home/kirrog/projects/CheckerApp/dataset/lenta-ru-news-only-text.txt"

In [ ]:
with open(clean_text_path, "w") as f:
    for line in r:
        f.write(f"{line}\n")

In [2]:
with open(clean_text_path, "r") as f:
    r = f.read().split("\n")

In [ ]:
r[0]

In [3]:
string_russian = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
augment_rate = 0.1


def augment_text(text: str):
    new_words = []
    words = text.split(" ")
    for w in words:
        if random.random() < augment_rate:
            pos = int((len(w) - 1) * random.random())
            try:
                if w[pos] in string_russian:
                    nw = w[:pos] + string_russian[int((len(string_russian) - 1) * random.random())] + w[pos + 1:]
                    w = nw
            except Exception as e:
                print(f"pos: {pos} word: {w} text: {text}")
        new_words.append(w)
    return " ".join(new_words)

In [4]:
data_augment = [augment_text(x) for x in tqdm(r)]

NameError: name 'tqdm' is not defined

In [6]:
augment_text_path = "/home/kirrog/projects/CheckerApp/dataset/lenta-ru-news-augment-text.txt"

In [ ]:
with open(augment_text_path, "w") as f:
    for line in data_augment:
        f.write(f"{line}\n")

In [7]:
with open(augment_text_path, "r") as f:
    data_augment = f.read().split("\n")

In [8]:
def slit_to_train_test_valid(data):
    tr, te = train_test_split(data, test_size=0.30, shuffle=False)
    test, valid = train_test_split(te, test_size=0.33, shuffle=False)
    return tr, test, valid

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
bert_tokens = {
    'START_SEQ': 101,
    'PAD': 0,
    'END_SEQ': 102,
    'UNK': 100
}

In [11]:
from typing import List


def fill_matrix_with_tokens(texts: List[str]) -> np.array:
    matrix = np.ones((len(texts), 768), dtype=np.int32) * bert_tokens["UNK"]
    counter = 0
    inp_size = 768
    for i, text_ in tqdm(enumerate(texts), desc="tokenizing", total=len(texts)):
        tokens = [bert_tokens["START_SEQ"]] + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_)) + [
            bert_tokens["END_SEQ"]]
        threshold = min(inp_size, len(tokens))
        if threshold > inp_size:
            counter+=1
        matrix[i, 0: threshold] = tokens[:threshold]
    print(f"Cutted: {counter}")
    return matrix


In [12]:
def save_as_train_test_valid_data(dir, prefix, data):
    train, test, valid = slit_to_train_test_valid(data)
    train_path = Path(dir) / (prefix + "_train.npy")
    test_path = Path(dir) / (prefix + "_test.npy")
    valid_path = Path(dir) / (prefix + "_valid.npy")
    np.save(str(train_path), fill_matrix_with_tokens(train))
    np.save(str(test_path), fill_matrix_with_tokens(test))
    np.save(str(valid_path), fill_matrix_with_tokens(valid))

In [13]:
len(r)

799743

In [14]:
r[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [15]:
save_as_train_test_valid_data("/home/kirrog/projects/CheckerApp/dataset", "clean", r)

tokenizing: 100%|██████████| 559820/559820 [04:20<00:00, 2148.12it/s]


Cutted: 0


tokenizing: 100%|██████████| 160748/160748 [01:49<00:00, 1467.04it/s]


Cutted: 0


tokenizing: 100%|██████████| 79175/79175 [00:56<00:00, 1407.38it/s]


Cutted: 0


In [ ]:
len(data_augment)

In [ ]:
data_augment[0]

In [16]:
save_as_train_test_valid_data("/home/kirrog/projects/CheckerApp/dataset", "aug", data_augment)

tokenizing: 100%|██████████| 559820/559820 [05:00<00:00, 1861.87it/s]


Cutted: 0


tokenizing: 100%|██████████| 160749/160749 [02:25<00:00, 1106.91it/s]


Cutted: 0


tokenizing: 100%|██████████| 79175/79175 [01:18<00:00, 1005.16it/s]


Cutted: 0


In [ ]:
d = np.zeros((10, 10), dtype=np.uint16)

In [ ]:
!pip install labml == 0.4.125

In [ ]:
!pip install transformers == 4.7.0

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
!cd / content / drive / MyDrive / punct & python / content / drive / MyDrive / punct / train.py \
    --model-name repunct-model \
    --pretrained-model DeepPavlov / rubert-base-cased-sentence \
    --targets S CM D EL EX Q CL \
    --train-data / content / drive / MyDrive / punct / lenta_res_train.tsv \
    --val-data / content / drive / MyDrive / punct / lenta_res_valid.tsv \
    --test-data / content / drive / MyDrive / punct / lenta_res_test.tsv \
    --store-best-weights \
    --epoch 7 \
    --batch-size 4 \
    --augment-rate 0.15 \
    --labml \
    --seed 1 \
    --cuda \
    --save-dir / content / drive / MyDrive / punct / models